# Deep learning model

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [10]:
# data loading from .npz generated from tcr_data_process.ipynb
# loading train data
npz = np.load('tcr_wop_train.npz')
train_inputs = npz['input'].astype(np.float)
train_targets = npz['targets'].astype(np.float)
# loading validation data
npz = np.load('tcr_wop_validation.npz')
validation_inputs, validation_targets = npz['input'].astype(np.float), npz['targets'].astype(np.float)

npz = np.load('tcr_wop_test.npz')
test_inputs, test_targets = npz['input'].astype(np.float), npz['targets'].astype(np.float)

C:\Users\parmarn\AppData\Local\Temp\ipykernel_14640\733935376.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs = npz['input'].astype(np.float)
C:\Users\parmarn\AppData\Local\Temp\ipykernel_14640\733935376.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_targets = npz['targets'].astype(np.float)
C:\Users\parmarn\Ap

# ANN Model

Outline, Optimizer, Loss Function, Early Stoping and Traing rate

In [11]:
# NN model prep
input_size =4
output_size = 2
hidden_layer_size = 10

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='linear')
])


In [12]:
# model prep, optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer, loss=tf.keras.losses.MeanSquaredError(),metrics=[tf.keras.metrics.RootMeanSquaredError()])


In [13]:
# Training
BATCH_SIZE = 1
MAX_EPOCHS = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

In [14]:
# fit the model
model.fit(
    train_inputs, train_targets, batch_size=BATCH_SIZE,epochs=MAX_EPOCHS, callbacks=[early_stopping], validation_data=(validation_inputs, validation_targets), verbose=2
          )

Epoch 1/100
49/49 - 2s - loss: 202.8214 - root_mean_squared_error: 14.2415 - val_loss: 101.6003 - val_root_mean_squared_error: 10.0797 - 2s/epoch - 37ms/step
Epoch 2/100
49/49 - 0s - loss: 195.5809 - root_mean_squared_error: 13.9850 - val_loss: 98.4942 - val_root_mean_squared_error: 9.9244 - 168ms/epoch - 3ms/step
Epoch 3/100
49/49 - 0s - loss: 190.4438 - root_mean_squared_error: 13.8001 - val_loss: 95.4902 - val_root_mean_squared_error: 9.7719 - 135ms/epoch - 3ms/step
Epoch 4/100
49/49 - 0s - loss: 185.8174 - root_mean_squared_error: 13.6315 - val_loss: 92.0741 - val_root_mean_squared_error: 9.5955 - 134ms/epoch - 3ms/step
Epoch 5/100
49/49 - 0s - loss: 180.4506 - root_mean_squared_error: 13.4332 - val_loss: 87.6624 - val_root_mean_squared_error: 9.3628 - 135ms/epoch - 3ms/step
Epoch 6/100
49/49 - 0s - loss: 172.9946 - root_mean_squared_error: 13.1527 - val_loss: 81.7631 - val_root_mean_squared_error: 9.0423 - 206ms/epoch - 4ms/step
Epoch 7/100
49/49 - 0s - loss: 161.7352 - root_mean_

# Model Test

In [15]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

1/1 [==============================] - 0s 282ms/step - loss: 2.7718 - root_mean_squared_error: 1.6649


In [16]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 2.77. Test accuracy: 166.49%


In [22]:
prediction = model.predict(test_inputs)
prediction = pd.DataFrame(prediction)
test_targets = pd.DataFrame(test_targets)

result = pd.concat([test_targets, prediction], axis = 1, ignore_index=True)
result.columns = ['Targets-dTi', 'Targets-dT','Prediction-dTi', 'Prediction-dT']

1/1 [==============================] - 0s 49ms/step


In [23]:
result

,Targets-dTi,Targets-dT,Prediction-dTi,Prediction-dT
0,12.0,0.333333,13.369974,1.417512
1,11.0,4.666667,8.352830,3.361921
2,14.0,6.666667,12.310569,4.709759
3,13.0,1.666667,12.957286,1.311367
4,14.0,11.333333,16.261000,13.719584
5,9.0,5.333333,10.736141,7.377071
6,16.0,0.666667,16.356882,2.117888
